In [6]:
#%matplotlib notebook
from scimple import scimple
#example :


molecTable=scimple.ImportTable("phenyl-Fe-porphyirine-CO2-Me_4_rel.xyz",firstLine=3,lastLine=103)
grapheneTable=scimple.ImportTable("phenyl-Fe-porphyirine-CO2-Me_4_rel.xyz",firstLine=104,lastLine=495)
chargesGraphene=scimple.ImportTable("CHARGES_phenyl-Fe-porphyirine-CO2-Me_4_rel",firstLine=104,lastLine=495)
print(molecTable.getTable())



[[None, 'H', 4.755162, -1.003886, 23.477489], [None, 'H', 7.188822, 2.987449, 23.504245], [None, 'H', 5.562981, -3.420502, 23.335969], [None, 'H', -3.719606, 4.115012, 23.264568], [None, 'H', -1.238212, 8.153534, 23.36198], [None, 'H', 9.694255, 3.470257, 23.371604], [None, 'H', 5.498527, 7.035964, 23.289556], [None, 'H', 0.337397, -1.41755, 23.344035], [None, 'H', 3.156778, 8.470036, 23.259472], [None, 'H', -2.005949, 0.009409, 23.309267], [None, 'H', -1.917773, 10.607451, 23.179509], [None, 'H', -6.220333, 3.664191, 23.036465], [None, 'C', 4.14609, -1.770662, 22.953253], [None, 'C', 4.499401, 6.698093, 22.956196], [None, 'C', 4.586434, -3.127127, 22.890068], [None, 'C', 0.180804, -0.379537, 22.995538], [None, 'C', 3.316105, 7.422865, 22.941106], [None, 'C', 7.56407, 3.874778, 22.950098], [None, 'C', -1.004801, 0.34217, 22.976939], [None, 'C', 8.962513, 4.156429, 22.888863], [None, 'H', -10.255206, 2.035537, 22.849338], [None, 'C', -4.080705, 3.192478, 22.762803], [None, 'H', -2.65889

In [7]:


#3D delta et molec

myPlot3D=scimple.CreatePlot(dim=3,xlabel="X",ylabel="Y",zlabel="Z",borders=[-40,40,-40,40,15,30],title="Test Graphe #3D delta et molec")
myPlot3D.addToPlot(molecTable,xColNum=2,yColNum=3,zColNum=4,markersize=2,coloredBy=1)
myPlot3D.addToPlot(grapheneTable,xColNum=2,yColNum=3,zColNum=4,markersize=2,label="graphene",coloredBy=lambda lineNum,line:(sum(chargesGraphene.getTable()[lineNum][1:]) -4))
scimple.show()


<IPython.core.display.Javascript object>

<div id='36f93380-3819-43cc-a45f-8348475fbc96'></div>

In [8]:
"""EN TESTS :
#3D molec avec couleurs standards
dicoCouleursStandards={'C':"#000000",'H':"#ffffff",'O':'r','N':'b','Fe':"#00ffff"}
myPlot3D=scimple.CreatePlot(dim=3,xlabel="X",ylabel="Y",zlabel="Z",borders=[-40,40,-40,40,15,30],title="Test Graphe #3D molec avec couleurs standards")
myPlot3D.addToPlot(molecTable,xColNum=2,yColNum=3,zColNum=4,coloredBy=lambda lineNum,line:dicoCouleursStandards[line[1]])
"""
#3D comparatif z et delta:

myPlot3Dbis=scimple.CreatePlot(dim=3,xlabel="X",ylabel="Y",zlabel="Z",borders=[-40,40,-40,40,15,30],title="Test Graphe #3D comparatif z et delta:")
myPlot3Dbis.addToPlot([grapheneTable.getTable()[i][:4]+[grapheneTable.getTable()[i][4]+10] for i in range (len(grapheneTable.getTable())-1)],xColNum=2,yColNum=3,zColNum=4,label="colored by z",coloredBy=lambda lineNum,line:line[4])
myPlot3Dbis.addToPlot(grapheneTable,xColNum=2,yColNum=3,zColNum=4,label="colored by delta",coloredBy=lambda lineNum,line:(sum(chargesGraphene.getTable()[lineNum][1:]) -4))
scimple.show()

<IPython.core.display.Javascript object>

<div id='be3f72f4-e613-4bdb-b2a2-1d19d0353ec7'></div>

In [10]:

#2D:

myPlot2D=scimple.CreatePlot(dim=2,xlabel="X",zlabel="Z",borders=[-20,20,18,19],title="Test Graphe 2D")
myPlot2D.addToPlot(grapheneTable,xColNum=3,yColNum=4,label="graphene Y/Z",coloredBy="#f4a28c",markersize=20)
myPlot2D.addToPlot(grapheneTable, xColNum=2,yColNum=4,label="graphene X/Z",plotType='-')
scimple.show()


<IPython.core.display.Javascript object>

<div id='f7cf70f6-862b-479e-ab92-ceb97eccabb5'></div>

In [9]:

#perso :
#deux surfaces :

myTable=scimple.ImportTable("ek_InTP_CO2_Me_4_graphene_W_r2_k.dat",firstLine=1)
 
myPlot3Dter=scimple.CreatePlot(dim=3,xlabel="X",ylabel="Y",zlabel="Z",title="deux surfaces, point de weyl ?")
myPlot3Dter.addToPlot(myTable,xColNum=0,yColNum=1,zColNum=4,label="column 4",coloredBy="#000000")
myPlot3Dter.addToPlot(myTable,xColNum=0,yColNum=1,zColNum=5,label="column 5")
scimple.show()


<IPython.core.display.Javascript object>

<div id='ba016a42-a073-4052-b16c-47b993f0a838'></div>

In [2]:
print(0)

0
